In [19]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import warnings
warnings.filterwarnings('ignore')

In [20]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.utils import multi_gpu_model

import keras_metrics
import pandas
import ast
import numpy as np


In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [22]:
train_set = "/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/train/"
validation_set = "/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/val/"
test_set = "/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/test/"

In [23]:
train_df = pandas.read_csv("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/train/output.csv")
valid_df = pandas.read_csv("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/val/output.csv")
test_df = pandas.read_csv("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/test/output.csv")

In [24]:
train_df.head(2)

,Unnamed: 0,image_id,x_1,y_1,width,height
0,139981,139982.jpg,114,138,159,220
1,172207,172208.jpg,86,42,112,155


In [25]:
BATCH_SIZE = 256
IMG_SIZE = 224
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.4,1.0])

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_set,
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=validation_set,
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 164105 validated image filenames.
Found 18234 validated image filenames.


In [26]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

In [27]:
model = Sequential()
model.add(InputLayer(input_shape=(224,224,3)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
          
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(2622, (1, 1)))

#model.add(Conv2D(filters=4, kernel_size=(1,1)))
#model.add(GlobalAveragePooling2D())
#model.add(BatchNormalization())

In [28]:
rule_4_neurons_number = [int(round(x/(i+1),0)) for (x,i) in zip([2622]*2,
                                           range(2))]

In [29]:
model.add(Flatten())
model.add(Dense(rule_4_neurons_number[0], activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(rule_4_neurons_number[1], activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(4))

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
batch_normalization_7 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 56, 56, 128)      

In [31]:
model = multi_gpu_model(model, gpus=2) #parallelize model

In [32]:
model.compile(optimizer=Adam(lr = 1e-5), loss='mse', metrics=["accuracy"])

In [33]:
model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)

72/71 [==============================] - 46s 638ms/step


[142759.9375, 0.0012065372429788113]

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='min')
mcp_save = ModelCheckpoint('yolf.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')

history = model.fit_generator(generator=train_generator, epochs=100, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, earlyStopping, reduce_lr_loss])

Epoch 1/100
642/641 [==============================] - 767s 1s/step - loss: 19165.6202 - accuracy: 0.7991 - val_loss: 17866.4941 - val_accuracy: 0.8098
Epoch 2/100
642/641 [==============================] - 788s 1s/step - loss: 16798.7198 - accuracy: 0.8092 - val_loss: 8826.1318 - val_accuracy: 0.8098
Epoch 3/100
642/641 [==============================] - 765s 1s/step - loss: 16229.5906 - accuracy: 0.8095 - val_loss: 19233.2578 - val_accuracy: 0.8098
Epoch 4/100
642/641 [==============================] - 767s 1s/step - loss: 15654.0811 - accuracy: 0.8095 - val_loss: 18899.7520 - val_accuracy: 0.8098
Epoch 5/100
642/641 [==============================] - 779s 1s/step - loss: 15072.4860 - accuracy: 0.8090 - val_loss: 6668.9395 - val_accuracy: 0.8072
Epoch 6/100
642/641 [==============================] - 779s 1s/step - loss: 14546.2964 - accuracy: 0.8072 - val_loss: 7542.3477 - val_accuracy: 0.7963
Epoch 7/100
642/641 [==============================] - 782s 1s/step - loss: 13937.6338 - ac

Epoch 53/100
642/641 [==============================] - 782s 1s/step - loss: 6512.6560 - accuracy: 0.8126 - val_loss: 5445.5586 - val_accuracy: 0.7897
Epoch 54/100
642/641 [==============================] - 776s 1s/step - loss: 6486.5202 - accuracy: 0.8126 - val_loss: 17790.7949 - val_accuracy: 0.7963
Epoch 55/100
642/641 [==============================] - 766s 1s/step - loss: 6490.3866 - accuracy: 0.8119 - val_loss: 15344.1807 - val_accuracy: 0.7947
Epoch 56/100
642/641 [==============================] - 769s 1s/step - loss: 6523.0344 - accuracy: 0.8125 - val_loss: 28550.1504 - val_accuracy: 0.7943
Epoch 57/100
642/641 [==============================] - 767s 1s/step - loss: 6687.9453 - accuracy: 0.8129 - val_loss: 4888.3965 - val_accuracy: 0.7935
Epoch 58/100
642/641 [==============================] - 777s 1s/step - loss: 6505.5056 - accuracy: 0.8120 - val_loss: 9131.7021 - val_accuracy: 0.7934
Epoch 59/100
642/641 [==============================] - 751s 1s/step - loss: 6484.6694 - ac

In [ ]:
print(history.history)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training accuracy')
plt.plot(val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.ylabel('accuracy')
plt.ylim([min(plt.ylim()),500])
plt.title('Training and Validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,20000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_set,
                                    x_col="image_id",
                                    y_col=["x_1","y_1","width","height"],
                                    class_mode="raw",
                                    shuffle=True,
                                    target_size=(224, 224),
                                    batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 


#CHANGE PARALLEL MODEL
pred=model.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)


In [ ]:
print(pred[1])

In [ ]:
from PIL import  Image, ImageDraw

count = 0

for gen_image in test_generator:
    img = Image.fromarray(np.uint8((gen_image[0][0])*255))
    img1 = ImageDraw.Draw(img)
    x1,y1 = pred[count][0],pred[count][1]
    x4,y4= pred[count][0] + pred[count][2], pred[count][1] + pred[count][3]
    img1.rectangle([(x1,y1),(x4,y4)], outline ="red") 
    #img.show() 
    img.save("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/results/output"+ str(count) + ".jpg")
    #print(gen_image[0][0].shape,pred[count])
    count = count + 1
    if count == 1000:
        break